### Pinecone Vector DB



In [ ]:
## Create your index and apikey from here https://www.pinecone.io/
# api_key="provide your apikey"

In [ ]:
# !pip install -qU langchain langchain-pinecone langchain-openai
import os
from dotenv import load_dotenv
load_dotenv()   

In [ ]:
from pinecone import Pinecone
api_key=os.getenv("PINECONE_API_KEY")
pc=Pinecone(api_key=api_key)

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small",dimensions=1024,api_key="openai_api_key")

In [ ]:
from pinecone import ServerlessSpec

index_name = "rag"  # change if desired

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [ ]:
index

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
vector_store

In [ ]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

load_dotenv()

# Explicitly pull the key from environment
api_key = os.getenv("OPENAI_API_KEY")

# Initialize embeddings with the key
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
    openai_api_key=api_key
)

# Initialize Vector Store
vector_store = PineconeVectorStore(
    index_name="your-index-name", 
    embedding=embeddings
)

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
key = os.getenv("OPENAI_API_KEY")

if key:
    print(f"Key found! Starts with: {key[:7]}...")
    if key.startswith("sk-"):
        print("Format looks correct.")
    else:
        print("Warning: Key does not start with 'sk-'. Check your .env file.")
else:
    print("No key found at all. Check your .env file location.")

In [ ]:
vector_store.add_documents(documents=documents)

In [ ]:
### Query Directly
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

In [ ]:
### Retriever
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.4},
)
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})